In [63]:
import pandas as pd
import numpy as np
import pickle
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder, OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler, PolynomialFeatures
from sklearn.compose import make_column_transformer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import warnings
from joblib import dump, load

In [2]:
train = pd.read_pickle("/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/liyan/train.pkl")
test = pd.read_pickle("/Users/liyannie/Desktop/Sp_2019_courses/DS_Indus/project/liyan/test.pkl")

In [3]:
X_train = train.loc[:, train.columns != 'travel_time']
y_train = train['travel_time']
X_test = test.loc[:, train.columns != 'travel_time']
y_test = test['travel_time']

## One hot encoding ( < 15 categories):
- mo
- fog
- rain_drizzle
- snow_ice_pellets
- day_of_week
- work
- weekday

## Target-based encoding ( >15 categories, + scaler for lin models):
- pickup_cluster
- dropoff_cluster
- hour_of_day

## Numerical (power transformation, + scaler for lin models?):
- passenger_count
- temp                
- visib                 
- wdsp                 
- gust                  
- max                   
- min                    
- prcp 
- num_rides_pcluster   
- num_rides_dcluster     
- num_rides_ccluster    
- haver_dist
- man_dist

### Other:
- pickup_longitude       
- pickup_latitude
- dropoff_longitude     
- dropoff_latitude

# 5.1 linear models

In [21]:
X_train_lin = X_train.drop(labels = ['pickup_longitude', 'pickup_latitude', 
                                     'dropoff_longitude', 'dropoff_latitude'], 
                           axis = 1).head(100000)
y_train_lin = y_train[:100000]

In [22]:
X_test_lin = X_test.drop(labels = ['pickup_longitude', 'pickup_latitude', 
                                     'dropoff_longitude', 'dropoff_latitude'], 
                           axis = 1)

## Ridge + OHE

In [541]:
# no preprocessing other than ohe
lr_ohe_feat1 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday',
           'pickup_cluster', 'dropoff_cluster', 'hour_of_day']
lr_pre1 = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', sparse = False) 
                                   lr_ohe_feat1),
                                  remainder='passthrough')

# !!! remainder = "passthrough" means we will keep these numerical variables that we did not specify in the column
# transformer. They will be appended to the end of the transformed matrix

In [542]:
ridge_pipe1 = make_pipeline(lr_pre1, Ridge())
ridge_params_grid1 = {'ridge__alpha': np.logspace(-3, 5, 10)}
ridge_grid1 = GridSearchCV(ridge_pipe1, ridge_params_grid1, cv=3, 
                           refit = 'neg_mean_squared_error', 
                           scoring = ['neg_mean_squared_error', 
                                     #'neg_mean_squared_log_error',
                                     'r2'])

In [543]:
%%time

warnings.filterwarnings('ignore')

ridge_grid1.fit(X_train_lin, y_train_lin)

CPU times: user 3min 7s, sys: 50.1 s, total: 3min 57s
Wall time: 3min 6s


In [544]:
# save model to joblib
dump(ridge_grid1, 'ridge_grid1.joblib')
ridge_grid1 = load('ridge_grid1.joblib') 

['ridge_grid1.joblib']

In [28]:
ridge_grid1 = load('ridge_grid1.joblib') 

In [545]:
ridge_grid1.best_params_

{'ridge__alpha': 215.44346900318823}

In [546]:
ridge_grid1.best_score_

-114796.94401367611

In [30]:
ridge_grid1.score(X_test_lin, y_test)

-210423.15805765736

In [548]:
warnings.filterwarnings('ignore')

ridge_result1 = pd.DataFrame(ridge_grid1.cv_results_)[['param_ridge__alpha', 'mean_test_neg_mean_squared_error', 
                                     #'mean_test_neg_mean_squared_log_error',
                                    'mean_test_r2']]
ridge_result1

,param_ridge__alpha,mean_test_neg_mean_squared_error,mean_test_r2
0,0.001,-120673.940163,0.667107
1,0.00774264,-120668.334166,0.667123
2,0.0599484,-120626.069251,0.667249
3,0.464159,-120346.209882,0.668082
4,3.59381,-118984.323682,0.672145
5,27.8256,-115409.613970,0.682875
6,215.443,-114796.944014,0.685004
7,1668.1,-118467.284651,0.674408
8,12915.5,-125498.874498,0.654685
9,100000,-131344.426444,0.639252


### Ridge + OHE + target encoder + polynomial (done - overfit)

In [72]:
ridge_ohe_feat2 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday']
ridge_target_feat2 = ['pickup_cluster', 'dropoff_cluster', 'hour_of_day']
ridge_pre2 = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', sparse = False), ridge_ohe_feat2),
                                     (TargetEncoder(handle_unknown = 'ignore'), ridge_target_feat2),
                                     remainder='passthrough')


In [73]:
ridge_pipe2 = make_pipeline(ridge_pre2, PolynomialFeatures(include_bias = False, interaction_only = True), Ridge())
ridge_params_grid2 = {'ridge__alpha': np.logspace(-3, 5, 10)}
ridge_grid2 = GridSearchCV(ridge_pipe2, ridge_params_grid2, cv=3, 
                           refit = 'neg_mean_squared_error', 
                           scoring = ['neg_mean_squared_error', 
                                     #'neg_mean_squared_log_error',
                                     'r2'])

In [74]:
%%time
warnings.filterwarnings('ignore')

ridge_grid2.fit(X_train_lin, y_train_lin)

CPU times: user 5min 40s, sys: 36.6 s, total: 6min 17s
Wall time: 5min 17s


In [75]:
# save model to joblib
dump(ridge_grid2, 'ridge_grid2.joblib')
#clf = load('filename.joblib') 

['ridge_grid2.joblib']

In [76]:
ridge_grid2.best_params_

{'ridge__alpha': 100000.0}

In [77]:
ridge_grid2.best_score_

-107022.57289050739

In [78]:
ridge_grid2.score(X_test_lin, y_test)

-199705.24330563875

In [79]:
warnings.filterwarnings('ignore')

ridge_result2 = pd.DataFrame(ridge_grid2.cv_results_)[['param_ridge__alpha', 'mean_test_neg_mean_squared_error', 
                                     #'mean_test_neg_mean_squared_log_error',
                                    'mean_test_r2']]
ridge_result2

,param_ridge__alpha,mean_test_neg_mean_squared_error,mean_test_r2
0,0.001,-121161.029377,0.669885
1,0.00774264,-121159.262048,0.669889
2,0.0599484,-121145.150226,0.669924
3,0.464159,-121036.655221,0.670189
4,3.59381,-120243.208333,0.672129
5,27.8256,-116037.989281,0.682421
6,215.443,-109400.858110,0.698757
7,1668.1,-108392.720412,0.701532
8,12915.5,-107218.233233,0.705185
9,100000,-107022.572891,0.705981


### Ridge + OHE + target encoder + (PowerTransformation + Scaling) for numerical

In [55]:
ridge_ohe_feat3 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday']
ridge_target_feat3 = ['pickup_cluster', 'dropoff_cluster', 'hour_of_day']
ridge_num_feat3 = ['passenger_count', 'temp', 'visib', 'wdsp', 'gust', 'max', 'min','prcp',
                  'haver_dist','man_dist','num_rides_pcluster','num_rides_dcluster', 'num_rides_ccluster']

ridge_num_pipe = make_pipeline(PowerTransformer(), StandardScaler())

ridge_pre3 = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', sparse = False), ridge_ohe_feat3),
                                     (TargetEncoder(handle_unknown = 'ignore'), ridge_target_feat3),
                                     (ridge_num_pipe, ridge_num_feat3))

In [56]:
ridge_pipe3 = make_pipeline(ridge_pre3, Ridge())
ridge_params_grid3 = {'ridge__alpha': np.logspace(-3, 5, 10)}
ridge_grid3 = GridSearchCV(ridge_pipe3, ridge_params_grid3, cv=3, 
                           refit = 'neg_mean_squared_error', 
                           scoring = ['neg_mean_squared_error', 
                                     #'neg_mean_squared_log_error',
                                     'r2'])

In [57]:
%%time
warnings.filterwarnings('ignore')

ridge_grid3.fit(X_train_lin, y_train_lin)

CPU times: user 1min 56s, sys: 7.01 s, total: 2min 3s
Wall time: 1min 3s


In [58]:
# save model to joblib
dump(ridge_grid3, 'ridge_grid3.joblib')
#clf = load('filename.joblib') 

['ridge_grid3.joblib']

In [59]:
ridge_grid3.best_params_

{'ridge__alpha': 27.825594022071257}

In [60]:
ridge_grid3.best_score_

-158773.8885575869

In [61]:
ridge_grid3.score(X_test_lin, y_test)

-237927.23336062572

In [62]:
warnings.filterwarnings('ignore')

ridge_result3 = pd.DataFrame(ridge_grid3.cv_results_)[['param_ridge__alpha', 'mean_test_neg_mean_squared_error', 
                                     #'mean_test_neg_mean_squared_log_error',
                                    'mean_test_r2']]
ridge_result3

,param_ridge__alpha,mean_test_neg_mean_squared_error,mean_test_r2
0,0.001,-158777.354005,0.563186
1,0.00774264,-158777.350685,0.563186
2,0.0599484,-158777.325023,0.563186
3,0.464159,-158777.129080,0.563186
4,3.59381,-158775.773964,0.563191
5,27.8256,-158773.888558,0.563203
6,215.443,-158989.425913,0.562654
7,1668.1,-160182.798775,0.559572
8,12915.5,-162912.950276,0.552904
9,100000,-200364.709866,0.450815


### Lasso + OHE + target encoder + (PowerTransformation + Scaling) for numerical

In [43]:
lasso_ohe_feat1 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday']
lasso_target_feat1 = ['pickup_cluster', 'dropoff_cluster', 'hour_of_day']
lasso_num_feat1 = ['passenger_count', 'temp', 'visib', 'wdsp', 'gust', 'max', 'min','prcp',
                  'haver_dist','man_dist','num_rides_pcluster','num_rides_dcluster', 'num_rides_ccluster']

lasso_num_pipe = make_pipeline(PowerTransformer(), StandardScaler())

lasso_pre1 = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', sparse = False), lasso_ohe_feat1),
                                     (TargetEncoder(handle_unknown = 'ignore'), lasso_target_feat1),
                                     (lasso_num_pipe, lasso_num_feat1))

In [47]:
lasso_pipe1 = make_pipeline(lasso_pre1, Lasso())
lasso_params_grid1 = {'lasso__alpha': np.logspace(-3, 5, 10)}
lasso_grid1 = GridSearchCV(lasso_pipe1, lasso_params_grid1, cv=3, 
                           refit = 'neg_mean_squared_error', 
                           scoring = ['neg_mean_squared_error', 
                                     #'neg_mean_squared_log_error',
                                     'r2'])

In [48]:
%%time
warnings.filterwarnings('ignore')

lasso_grid1.fit(X_train_lin, y_train_lin)

CPU times: user 3min 7s, sys: 8.45 s, total: 3min 15s
Wall time: 1min 39s


In [49]:
# save model to joblib
dump(lasso_grid1, 'lasso_grid1.joblib')
#clf = load('filename.joblib') 

['lasso_grid1.joblib']

In [50]:
lasso_grid1.best_params_

{'lasso__alpha': 0.46415888336127775}

In [51]:
lasso_grid1.best_score_

-158559.9683848672

In [52]:
lasso_grid1.score(X_test_lin, y_test)

-242776.5468209388

In [54]:
warnings.filterwarnings('ignore')

lasso_result1 = pd.DataFrame(lasso_grid1.cv_results_)[['param_lasso__alpha', 'mean_test_neg_mean_squared_error', 
                                     #'mean_test_neg_mean_squared_log_error',
                                    'mean_test_r2']]
lasso_result1

,param_lasso__alpha,mean_test_neg_mean_squared_error,mean_test_r2
0,0.001,-158776.878847,0.563187
1,0.00774264,-158773.603925,0.563198
2,0.0599484,-158731.735357,0.563324
3,0.464159,-158559.968385,0.563885
4,3.59381,-159519.732475,0.561719
5,27.8256,-162581.508054,0.553798
6,215.443,-207418.558196,0.431612
7,1668.1,-363455.708787,-0.000784
8,12915.5,-363455.708787,-0.000784
9,100000,-363455.708787,-0.000784


## 5.2 Tree-based ensemble models

In [81]:
X_train_tr = X_train.drop(labels = ['pickup_longitude', 'pickup_latitude', 
                                     'dropoff_longitude', 'dropoff_latitude'], 
                           axis = 1).tail(100000)
y_train_tr = y_train.tail(100000)

In [82]:
X_train_tr.columns

Index(['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'mo', 'temp', 'visib', 'wdsp',
       'gust', 'max', 'min', 'prcp', 'fog', 'rain_drizzle', 'snow_ice_pellets',
       'haver_dist', 'man_dist', 'hour_of_day', 'day_of_week', 'work',
       'weekday', 'pickup_cluster', 'dropoff_cluster', 'num_rides_pcluster',
       'num_rides_dcluster', 'num_rides_ccluster'],
      dtype='object')

### Random Forest + OHE + TargetEncoder (done)

In [35]:
rf_ohe_feat1 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday']
rf_target_feat1 = ['pickup_cluster', 'dropoff_cluster', 'hour_of_day']

rf_pre1 = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore', sparse = False), rf_ohe_feat1),
    (TargetEncoder(handle_unknown = 'ignore'), rf_target_feat1),
    remainder='passthrough')

In [36]:
# fit random forest
rf_pipe1 = make_pipeline(rf_pre1, RandomForestRegressor())
rf_params_grid1 = {'randomforestregressor__n_estimators': [100, 200, 300],
                  'randomforestregressor__max_features': [0.7, 0.8, 0.9]}
rf_grid1 = GridSearchCV(rf_pipe1, rf_params_grid1, cv=3, refit = 'neg_mean_squared_error', n_jobs = -1,
                       scoring = ['neg_mean_squared_error', 
                                  'neg_mean_squared_log_error',
                                  'r2'])

In [37]:
%%time

rf_grid1.fit(X_train_tr, y_train_tr)

CPU times: user 3min 24s, sys: 2.4 s, total: 3min 27s
Wall time: 34min 50s


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('onehotencoder', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignor...s='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomforestregressor__n_estimators': [100, 200, 300], 'randomforestregressor__max_features': [0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit='neg_mean_squared_error',
       return_train_score='warn',
       scoring=['neg_mean_squared_error', 'neg_mean_squared_log_error', 'r2'],
       verbose=0)

In [38]:
# save model to joblib
dump(rf_grid1, 'rf_grid1.joblib')
#clf = load('filename.joblib') 

['rf_grid1.joblib']

In [39]:
rf_grid1.best_params_

{'randomforestregressor__max_features': 0.7,
 'randomforestregressor__n_estimators': 200}

In [40]:
rf_grid1.best_score_

-108311.57550786324

In [41]:
warnings.filterwarnings('ignore')

rf_result1 = pd.DataFrame(rf_grid1.cv_results_)[['param_randomforestregressor__max_features', 
                                                'param_randomforestregressor__n_estimators', 
                                                'mean_test_neg_mean_squared_error', 
                                                'mean_test_neg_mean_squared_log_error',
                                                'mean_test_r2']]
rf_result1

,param_randomforestregressor__max_features,param_randomforestregressor__n_estimators,mean_test_neg_mean_squared_error,mean_test_neg_mean_squared_log_error,mean_test_r2
0,0.7,100,-108780.106365,-0.158181,0.756900
1,0.7,200,-108311.575508,-0.157426,0.757906
2,0.7,300,-108613.121181,-0.158415,0.757266
3,0.8,100,-109996.965066,-0.158432,0.753968
4,0.8,200,-110027.729652,-0.161279,0.754268
5,0.8,300,-109243.902910,-0.158631,0.755779
6,0.9,100,-111002.409712,-0.160654,0.751899
7,0.9,200,-110416.212409,-0.159462,0.753179
8,0.9,300,-109712.374797,-0.158679,0.754674


### XGboost + only OHE (done)

In [653]:
# no preprocessing other than ohe
tr_ohe_feat1 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday',
                'pickup_cluster', 'dropoff_cluster', 'hour_of_day']
tr_pre1 = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', sparse = False), tr_ohe_feat1),
                                 remainder='passthrough')

In [654]:
xg_pipe1 = make_pipeline(tr_pre1, XGBRegressor())
xg_params_grid1 = {'xgbregressor__max_depth': [10, 15, 20],
                   'xgbregressor__min_child_weight': [10, 20, 30]}
xg_grid1 = GridSearchCV(xg_pipe1, xg_params_grid1, cv=3, refit = 'neg_mean_squared_error', n_jobs = -1,
                       scoring = ['neg_mean_squared_error', 
                                  #'neg_mean_squared_log_error',
                                  'r2'])

In [655]:
%%time
xg_grid1.fit(X_train_tr, y_train_tr)

CPU times: user 3min 41s, sys: 2.66 s, total: 3min 44s
Wall time: 39min 6s


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('onehotencoder', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignor... reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'xgbregressor__max_depth': [10, 15, 20], 'xgbregressor__min_child_weight': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit='neg_mean_squared_error',
       return_train_score='warn', scoring=['neg_mean_squared_error', 'r2'],
       verbose=0)

In [656]:
# save model to joblib
dump(xg_grid1, 'xg_grid1.joblib')
#clf = load('filename.joblib') 

['xg_grid1.joblib']

In [657]:
xg_grid1.best_params_

{'xgbregressor__max_depth': 15, 'xgbregressor__min_child_weight': 30}

In [658]:
xg_grid1.best_score_

-106615.32371894142

In [659]:
xg_grid1.score(X_test, y_test)

-154107.97800224752

In [660]:
warnings.filterwarnings('ignore')

xg_result1 = pd.DataFrame(xg_grid1.cv_results_)[['param_xgbregressor__max_depth', 
                                                'param_xgbregressor__min_child_weight', 
                                                #'mean_test_neg_mean_squared_error', 
                                                'mean_test_neg_mean_squared_error',
                                                'mean_test_r2']]
xg_result1

,param_xgbregressor__max_depth,param_xgbregressor__min_child_weight,mean_test_neg_mean_squared_error,mean_test_r2
0,10,10,-107570.437095,0.759445
1,10,20,-107485.455627,0.760027
2,10,30,-107787.477562,0.759484
3,15,10,-108679.835036,0.756963
4,15,20,-107469.899916,0.759830
5,15,30,-106615.323719,0.761689
6,20,10,-109788.689964,0.754586
7,20,20,-108476.000347,0.757508
8,20,30,-107283.524675,0.760330


### XGboost + OHE + target encoder (done)

In [4]:
tr_ohe_feat2 = ['mo', 'fog', 'rain_drizzle', 'snow_ice_pellets', 'day_of_week', 'work', 'weekday']
tr_target_feat2 = ['pickup_cluster', 'dropoff_cluster', 'hour_of_day']

tr_pre2 = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore', sparse = False), tr_ohe_feat2),
    (TargetEncoder(handle_unknown = 'ignore'), tr_target_feat2),
    remainder='passthrough')

In [8]:
xg_pipe2 = make_pipeline(tr_pre2, XGBRegressor())
xg_params_grid2 = {'xgbregressor__max_depth': [15, 10, 25],
              'xgbregressor__min_child_weight': [10, 20, 30]}
xg_grid2 = GridSearchCV(xg_pipe2, xg_params_grid2, cv=3, n_jobs = -1,
                        refit = 'neg_mean_squared_error', 
                        scoring = ['neg_mean_squared_error', 
                                  #'neg_mean_squared_log_error',
                                  'r2'])

In [11]:
%%time
xg_grid2.fit(X_train_tr, y_train_tr)

/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Applications/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


CPU times: user 40.2 s, sys: 906 ms, total: 41.1 s
Wall time: 11min


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('onehotencoder', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignor... reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'xgbregressor__max_depth': [15, 10, 25], 'xgbregressor__min_child_weight': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit='neg_mean_squared_error',
       return_train_score='warn', scoring=['neg_mean_squared_error', 'r2'],
       verbose=0)

In [12]:
# save model to joblib
dump(xg_grid2, 'xg_grid2.joblib')
#clf = load('filename.joblib') 

['xg_grid2.joblib']

In [13]:
xg_grid2.best_params_

{'xgbregressor__max_depth': 10, 'xgbregressor__min_child_weight': 30}

In [14]:
xg_grid2.best_score_

-102368.98015886488

In [15]:
xg_grid2.score(X_test, y_test)

-140185.9996590946

In [16]:
warnings.filterwarnings('ignore')

xg_result2 = pd.DataFrame(xg_grid2.cv_results_)[['param_xgbregressor__max_depth', 
                                                'param_xgbregressor__min_child_weight', 
                                                #'mean_test_neg_mean_squared_error', 
                                                'mean_test_neg_mean_squared_error',
                                                'mean_test_r2']]
xg_result2

,param_xgbregressor__max_depth,param_xgbregressor__min_child_weight,mean_test_neg_mean_squared_error,mean_test_r2
0,15,10,-106362.352181,0.762192
1,15,20,-104077.776087,0.767168
2,15,30,-102992.388587,0.769598
3,10,10,-103924.321197,0.767564
4,10,20,-103218.794775,0.769149
5,10,30,-102368.980159,0.771106
6,25,10,-108091.970081,0.758372
7,25,20,-105046.798502,0.764987
8,25,30,-102952.495024,0.769606


# Best model among the 7 is the XGBoost + OHE + Target Encoding. Will use this for predictions